# Thema: Organisations-IDs für wissenschaftliche Einrichtungen anhand von Emailadressen gewinnen

### 1 Fragestellung

Die Aufgabe bestand darin, die Emailadressen den entsprechenden Institutionen zuzuordnen.

Die Datenquellen dafür waren:

- hybrid_publications.csv https://github.com/subugoe/hybrid_oa_dashboard/blob/master/data/hybrid_publications.csv?raw=true

    (Tabelle zu hybriden OA-Artikeln mit Angaben zu Lizenz, Verlag, DOI des OA-Artikels, Erscheinungsjahr, jährl. Umfang der Zeitschrift uvm wie Host, Subdomain, Domain und Suffix, erstellt mit dem Analysewerkzeug Hybrid OA Dashboard)
    

- GRID Open Data Dump https://figshare.com/articles/GRID_release_2018-11-14/7346852

 (Darin sind Tabellen mit Informationen zu Institutionen, die jew. durch ihre Grid-Id identifiziert werden, sowie Akronyme, Adressen, andere Bezeichnungen für die Institutionen, weitere Ids, uvm. wie z.B. eine Datei, in der das Verhältnis der verschiedenen Institutionen zueinander verzeichnet ist, z.B. Institute, die ihrer Universität untergeordnet sind.)

Außerdem wurde die Datei guete_bearbeitet.csv verwendet, die im Rahmen unserer ersten Aufgabe zur Güte von Recall und Precision erstellt wurde.

### 2 Vorgehen

Aufgabe: Matching domains mit GRID Webseiten-URLs

Tipp: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrlame.join.html

Die Grundidee ist, mithilfe der Programmierbibliothek Pandas von Python passende Spalten aus den vielen verschiedenen Datenquellen per „Join“ zusammenzustellen. So soll man am Ende die Emailadressen den Institutionen zuordnen können.

In [1]:
#!pip install pandas
import pandas as pd

In [2]:
hybrid_publications = pd.read_csv('hybrid_publications.csv', usecols=['host', 'domain', 'suffix'])
bearbeitet = pd.read_csv('guete_bearbeitet.csv', index_col = 0)
institutes = pd.read_csv('institutes.csv', usecols=['grid_id','name'])
links = pd.read_csv('links.csv')
relationships = pd.read_csv('relationships.csv')

In [3]:
print(hybrid_publications.head())

             host    domain  suffix
0  bradford.ac.uk  bradford   ac.uk
1         upol.cz      upol      cz
2       uwe.ac.uk       uwe   ac.uk
3        sina.com      sina     com
4     dlmu.edu.cn      dlmu  edu.cn


In [4]:
print(bearbeitet.head())

              email                                            tdm_url  \
0  meduniwien.ac.at  https://api.elsevier.com/content/article/PII:S...   
1        bruker.com  http://link.springer.com/article/10.1007/s1183...   
2          ohsu.edu  https://api.elsevier.com/content/article/PII:S...   
3           cox.net                                                      
4        amc.uva.nl  http://link.springer.com/article/10.1007/s1176...   

                time_stamp    mine_method                           doi  \
0  2018-12-17 16:06:12 UTC   Elsevier XML   10.1016/j.ddtec.2014.03.009   
1  2018-12-18 08:00:29 UTC  Springer HTML     10.1007/s11837-018-2752-0   
2  2018-12-17 16:06:12 UTC   Elsevier XML  10.1016/j.ocemod.2015.12.007   
3  2018-12-16 12:29:19 UTC  PMC OAI (NIH)     10.1007/s13752-016-0244-4   
4  2019-01-04 14:40:57 UTC  Springer HTML     10.1007/s11764-018-0690-z   

   recall  precision comment  
0    True       True     NaN  
1    True       True     NaN  
2    True  

In [5]:
print(institutes.head())

       grid_id                            name
0  grid.1001.0  Australian National University
1  grid.1002.3               Monash University
2  grid.1003.2        University of Queensland
3  grid.1004.5            Macquarie University
4  grid.1005.4                  UNSW Australia


In [6]:
print(links.head())

       grid_id                      link
0  grid.1001.0    http://www.anu.edu.au/
1  grid.1002.3    http://www.monash.edu/
2  grid.1003.2     http://www.uq.edu.au/
3  grid.1004.5         http://mq.edu.au/
4  grid.1005.4  https://www.unsw.edu.au/


In [7]:
print(relationships.head())

       grid_id relationship_type related_grid_id
0  grid.1001.0           Related   grid.460694.9
1  grid.1001.0           Related   grid.413314.0
2  grid.1001.0           Related   grid.460721.6
3  grid.1002.3           Related     grid.1623.6
4  grid.1002.3           Related   grid.414580.c


Zunächst wurden die Dateien links und institutes" durch "merge" basierend auf der beiden gemeinsamen Spalte "grid_id" zusammengefügt in der neuen Datei name_and_link.

In [8]:
name_and_link = institutes.merge(links, on='grid_id')

In [9]:
name_and_link_and_relationships = name_and_link.merge(relationships, on='grid_id')

In [10]:
print(name_and_link.head())

       grid_id                            name                      link
0  grid.1001.0  Australian National University    http://www.anu.edu.au/
1  grid.1002.3               Monash University    http://www.monash.edu/
2  grid.1003.2        University of Queensland     http://www.uq.edu.au/
3  grid.1004.5            Macquarie University         http://mq.edu.au/
4  grid.1005.4                  UNSW Australia  https://www.unsw.edu.au/


In [11]:
print(name_and_link_and_relationships.head())

       grid_id                            name                    link  \
0  grid.1001.0  Australian National University  http://www.anu.edu.au/   
1  grid.1001.0  Australian National University  http://www.anu.edu.au/   
2  grid.1001.0  Australian National University  http://www.anu.edu.au/   
3  grid.1002.3               Monash University  http://www.monash.edu/   
4  grid.1002.3               Monash University  http://www.monash.edu/   

  relationship_type related_grid_id  
0           Related   grid.460694.9  
1           Related   grid.413314.0  
2           Related   grid.460721.6  
3           Related     grid.1623.6  
4           Related   grid.414580.c  


Mithilfe der Funktionen extract_domain und extract_suffix konnten Bestandteile wie "http://", "www" und "/"  aus der Datei herausgefiltert werden. 

In [12]:
def extract_domain(link):
    link = str(link)
    link = link.replace('http://', '')
    link = link.replace('https://', '')
    first_position = link.find('www')
    last_position = link.find('/')
    link = link[first_position+4:last_position]
    link = link.split('.')
    domain = link[0]
    return str(domain)

def extract_suffix(link):
    link = str(link)
    link = link.replace('http://', '')
    link = link.replace('https://', '')
    first_position = link.find('www')
    if first_position == -1:
        try:
            first_position = link[0::]
        except:
            pass
    else:
        try:
            first_position = link.find('www')
        except:
            pass
    last_position = link.find('/')
    if last_position == -1:
        try:
            link = link[first_position+4::]
        except: 
            pass
    else:
        try:
            link = link[first_position+4:last_position]
        except:
            pass
    link = link.split('.')
    suffix_list = []
    for element in link:
        suffix_list.append(element)
    suffix_list = suffix_list[1::]
    suffix = ""
    for element in suffix_list:
        suffix = suffix + "." + str(element)
    return str(suffix[1::])

In [13]:
print(extract_suffix('http://www.kek.ac.de/'))

ac.de


In [14]:
print(extract_suffix('www.amu.edu.pl/'))

edu.pl


In [15]:
kek = 'amu.edu.pl'
print(kek.find('www'))

-1


Dann wurden auch name_and_link mit hybrid_publications basierend auf den Spalten "suffix" zusammengefügt zur Datei merge_link_and_host. 

In [16]:
name_and_link['domain'] = name_and_link['link'].apply(extract_domain)
name_and_link['suffix'] = name_and_link['link'].apply(extract_suffix)

In [17]:
merge_link_and_host = hybrid_publications.merge(name_and_link, on=['domain', 'suffix'])

In [18]:
print(merge_link_and_host.head())

             host    domain suffix      grid_id                    name  \
0  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   
1  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   
2  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   
3  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   
4  bradford.ac.uk  bradford  ac.uk  grid.6268.a  University of Bradford   

                                  link  
0  http://www.bradford.ac.uk/external/  
1  http://www.bradford.ac.uk/external/  
2  http://www.bradford.ac.uk/external/  
3  http://www.bradford.ac.uk/external/  
4  http://www.bradford.ac.uk/external/  


Durch "drop.duplicates" wurden dann alle Doppelungen entfernt.

In [19]:
merge_link_and_host = merge_link_and_host.drop_duplicates().reset_index(drop=True)

In [20]:
print(merge_link_and_host.head())

                     host    domain suffix        grid_id  \
0          bradford.ac.uk  bradford  ac.uk    grid.6268.a   
1  student.bradford.ac.uk  bradford  ac.uk    grid.6268.a   
2                 upol.cz      upol     cz  grid.10979.36   
3           oushi.upol.cz      upol     cz  grid.10979.36   
4          optics.upol.cz      upol     cz  grid.10979.36   

                          name                                 link  
0       University of Bradford  http://www.bradford.ac.uk/external/  
1       University of Bradford  http://www.bradford.ac.uk/external/  
2  Palacký University, Olomouc               http://www.upol.cz/en/  
3  Palacký University, Olomouc               http://www.upol.cz/en/  
4  Palacký University, Olomouc               http://www.upol.cz/en/  


In [21]:
merge_link_and_host.to_csv('test.csv')

Anschließend wurde merge_link_and_host mit bearbeitet (Ergebnis der ersten Aufgabe zu Güte von Recall und Precision) zur neuen Datei join_host_and_bearbeitet zusammengefügt.

In [22]:
join_host_and_bearbeitet = pd.merge(bearbeitet, merge_link_and_host, how='left', left_on='email', right_on='host')

In [23]:
print(join_host_and_bearbeitet.head())

              email                                            tdm_url  \
0  meduniwien.ac.at  https://api.elsevier.com/content/article/PII:S...   
1  meduniwien.ac.at  https://api.elsevier.com/content/article/PII:S...   
2  meduniwien.ac.at  https://api.elsevier.com/content/article/PII:S...   
3        bruker.com  http://link.springer.com/article/10.1007/s1183...   
4        bruker.com  http://link.springer.com/article/10.1007/s1183...   

                time_stamp    mine_method                          doi  \
0  2018-12-17 16:06:12 UTC   Elsevier XML  10.1016/j.ddtec.2014.03.009   
1  2018-12-17 16:06:12 UTC   Elsevier XML  10.1016/j.ddtec.2014.03.009   
2  2018-12-17 16:06:12 UTC   Elsevier XML  10.1016/j.ddtec.2014.03.009   
3  2018-12-18 08:00:29 UTC  Springer HTML    10.1007/s11837-018-2752-0   
4  2018-12-18 08:00:29 UTC  Springer HTML    10.1007/s11837-018-2752-0   

   recall  precision comment              host      domain suffix  \
0    True       True     NaN  meduniwien.

Schließlich wurden die Werte in dieser Datei noch anhand der Emailadressen sortiert.

In [24]:
join_host_and_bearbeitet = join_host_and_bearbeitet.sort_values(by=['email']).reset_index(drop=True)

In [25]:
join_host_and_bearbeitet.to_csv('merge.csv')

In [26]:
show_result = pd.read_csv('merge.csv', index_col = 0, usecols = ['email','name', 'grid_id'])

### 3 Ergebnisse

Viele E-Mailadressen konnten korrekt zugeordnet werden.

Problematisch für das Vorgehen sind logischerweise vor allem die Fälle, in denen die Forscher keine Emailadressen ihrer Institution angegeben haben, sondern Gmail, Yahoo oder ähnliches.

In [27]:
print(show_result)

                          grid_id                                      name
email                                                                      
                              NaN                                       NaN
                              NaN                                       NaN
                              NaN                                       NaN
                              NaN                                       NaN
                              NaN                                       NaN
126.com                       NaN                                       NaN
Outlook.com                   NaN                                       NaN
aau.in              grid.411373.3             Anand Agricultural University
adm.cgmh.org.tw     grid.413801.f              Chang Gung Memorial Hospital
adm.cgmh.org.tw     grid.413804.a    Kaohsiung Chang Gung Memorial Hospital
adm.cgmh.org.tw     grid.454209.e      Keelung Chang Gung Memorial Hospital
adm.cgmh.org

Anschließend wurde das Verfahren unter Einbeziehung der relationships-Datei wiederholt:

In [28]:
name_and_link_and_relationships['domain'] = name_and_link_and_relationships['link'].apply(extract_domain)
name_and_link_and_relationships['suffix'] = name_and_link_and_relationships['link'].apply(extract_suffix)
merge_link_and_host_and_relationships = hybrid_publications.merge(name_and_link_and_relationships, on=['domain', 'suffix'])
merge_link_and_host_and_relationships = merge_link_and_host_and_relationships.drop_duplicates().reset_index(drop=True)
merge_link_and_host_and_relationships.to_csv('test_with_relationships.csv')
join_host_and_bearbeitet_and_relationships = pd.merge(bearbeitet, merge_link_and_host_and_relationships, how='left', left_on='email', right_on='host')
join_host_and_bearbeitet_relationships = join_host_and_bearbeitet_and_relationships.sort_values(by=['email']).reset_index(drop=True)
join_host_and_bearbeitet_and_relationships.to_csv('merge_with_relationships.csv')
show_result_with_relationships = pd.read_csv('merge_with_relationships.csv', index_col = 0, usecols = ['email','name', 'grid_id', 'relationship_type', 'related_grid_id'])

In [29]:
print(show_result_with_relationships)

                        grid_id  \
email                             
meduniwien.ac.at  grid.22937.3d   
meduniwien.ac.at  grid.22937.3d   
meduniwien.ac.at  grid.22937.3d   
meduniwien.ac.at  grid.22937.3d   
meduniwien.ac.at  grid.22937.3d   
meduniwien.ac.at  grid.22937.3d   
meduniwien.ac.at  grid.22937.3d   
meduniwien.ac.at  grid.22937.3d   
meduniwien.ac.at  grid.454395.a   
meduniwien.ac.at  grid.476556.3   
bruker.com        grid.423218.e   
bruker.com        grid.423270.0   
bruker.com        grid.423270.0   
bruker.com        grid.423270.0   
bruker.com        grid.423270.0   
bruker.com        grid.423270.0   
bruker.com        grid.423270.0   
bruker.com        grid.461847.8   
bruker.com        grid.473958.4   
bruker.com        grid.481597.6   
bruker.com        grid.481598.9   
ohsu.edu            grid.5288.7   
ohsu.edu            grid.5288.7   
ohsu.edu            grid.5288.7   
ohsu.edu            grid.5288.7   
ohsu.edu            grid.5288.7   
ohsu.edu            

### 4 Diskussion

Man könnte noch die Angaben in der Datei „relationships.csv“ miteinbeziehen, um Institutionen, die anderen untergeordnet sind und evtl. die gleiche oder erweiterte Emailadressen verwenden, korrekt zuzuordnen. In der Tabelle merge_with_relationships.csv sind die in Beziehung stehenden Institutionen zwar verzeichnet, aber nur mit ihrer Grid-Id, nicht mit Namen. Dazu haben wir noch keine Lösung.

Bei einigen nicht erfolgten Zuordnungen liegt es daran, dass eine subdomain vorhanden ist, die wir noch nicht extrahieren und miteinbeziehen konnten.

In anderen Fällen konnten wir uns leider nicht erklären, warum keine korrekte Zuordnung erfolgt ist.

Zur Lösung dieser Probleme müsste der Algorithmus bei den extract-Funktionen noch verfeinert werden.

### 5 Fazit

Insgesamt funktioniert das Vorgehen ganz gut, sodass relativ viele Emailadressen korrekt zugeordnet werden können.